In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage,MultiModalMessage
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("GEMINI_API_KEY")

In [2]:
model_client=OpenAIChatCompletionClient(
  model="gemini-1.5-flash-8b",
  api_key=api_key
)

In [3]:
def web_search():
  "you are a golden retirver tool"
  return "The golden retriver is a dog breed from another country"

In [19]:
agent=AssistantAgent(
  name="webagent",
  model_client=model_client,
  system_message="use tool to ansewr the user query",
  tools=[web_search]
)

In [20]:
response=await agent.run(task="tell me about golden retriver")
response.messages[-1].content

'The golden retriver is a dog breed from another country'

In [21]:
from autogen_core import CancellationToken

async def check_tool()->None:
  response = await agent.on_messages(
    messages=[TextMessage(content="Tell me about golden retriver",source='user')],
    cancellation_token=CancellationToken()
  )

  print(response.inner_messages)
  print('\n\n\n')
  print(response.chat_message)


In [22]:
await check_tool()

[ToolCallRequestEvent(id='31cb7247-9402-443e-a9e8-22b56ce48d9c', source='webagent', models_usage=RequestUsage(prompt_tokens=48, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 7, 27, 14, 3, 48, 829076, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='', arguments='{}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='14330d0b-5503-498e-aae0-2e4b5aa931bd', source='webagent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 14, 3, 48, 829076, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The golden retriver is a dog breed from another country', name='web_search', call_id='', is_error=False)], type='ToolCallExecutionEvent')]




id='aaf50348-f080-4c2d-af7f-a813c2aafdfc' source='webagent' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 27, 14, 3, 48, 829076, tzinfo=datetime.timezone.utc) content='The golden retriver is a dog breed from another country' type='To

## Streaming Messages

In [24]:
from autogen_agentchat.ui import Console

async def console_output():
  await Console(
    agent.on_messages_stream(
      messages=[TextMessage(content="Tell me about golden retriver",source='user')],
      cancellation_token=CancellationToken()
    ),
    output_stats=True
  )

await console_output()

---------- ToolCallRequestEvent (webagent) ----------
[FunctionCall(id='', arguments='{}', name='web_search')]
[Prompt tokens: 72, Completion tokens: 3]
---------- ToolCallExecutionEvent (webagent) ----------
[FunctionExecutionResult(content='The golden retriver is a dog breed from another country', name='web_search', call_id='', is_error=False)]
---------- webagent ----------
The golden retriver is a dog breed from another country
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 72
Total completion tokens: 3
Duration: 2.63 seconds
